In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
!cat '/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/train/HS/1002.txt'

RT @USER: خلاص يا استيعابي يا جزمه بكره اختبار deal with itايش حركات الإجازه هذي خلاص عاد	OFF	NOT_HS


In [ ]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/dev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 7000 files belonging to 2 classes.
Using 5600 files for training.
Found 7000 files belonging to 2 classes.
Using 1400 files for validation.
Found 3470 files belonging to 2 classes.
Number of batches in raw_train_ds: 175
Number of batches in raw_val_ds: 44
Number of batches in raw_test_ds: 109


In [ ]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i].decode('utf-8').strip())
        print(label_batch.numpy()[i])
        print('--------------------------------')

عندي مشكلة كبيرة مع الناس اللي تشرحلهاا انت ايه وعيوبك ايه ومابتحبش ايه فايعملوا كل اللي انت حذرتهم منه ومايتخيلوش رد فعلك ويقولوا اتصدمناا فيك، ياعالم يا جزم يا ولاد الافاعي اومال اناا كنت بشرح في ايه من الاول	OFF	NOT_HS
0
--------------------------------
@USER عاااااااااااااااااااااااااااااااش يا ميرووو يا ميرا يا ميرتااااا احبك❤️❤️❤️😭	NOT_OFF	NOT_HS
1
--------------------------------
المشكلة في البلد دي إنك لو قلت للحلو يا حلو في عيونه يا إما بياخد قلم في نفسه أو بيخزقلك عيونك..!	NOT_OFF	NOT_HS
1
--------------------------------
#اطمن_انت_مش_لوحدك<LF>الخرفان انتخت الي الابد<LF>حاسس بيك يا مهتز يا ابن المط..رر.الاقبال النهاردة كان تاريخي لا نفع معاهم اطمن و لا بلالين و لا كتابة علي الحيطان و لا الفلوس و لا الحلل و لا الغطيان <LF>يا قهرتك يا مهتز<LF>يا فضيحتكم يا خرفان<LF>الخرفان هيتجننو <LF>السيسي هينفخكم ١٠ سنين اني هههه	OFF	HS
0
--------------------------------
@USER يا زيدي يا زيدي <LF>ايه الشعر ده المتنبي في زمانه<LF>هههههههه	NOT_OFF	NOT_HS
1
--------------------------------


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<LF>", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )

max_features = 20000
embedding_dim = 128
sequence_length = 500


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)




In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## Build a model 

In [ ]:
from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model.fit(train_ds, epochs=10,
                    validation_data=val_ds,
                    validation_steps=30)

Epoch 1/10
175/175 [==============================] - ETA: 0s - loss: 0.5485 - accuracy: 0.6164

## Train the model

In [ ]:
# epochs = 3

# # Fit the model using the train and test datasets.
# history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

## Evaluate the model on the test set


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

## plotlib

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

#epochs_range = range(22)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()